In [1]:
import numpy as np
import torch

In [2]:
train_on_gpu = torch.cuda.is_available()


if not train_on_gpu:
    print('cuda is not available,   training on cpu')
else:
    print('cuda is available,    training on gpu')

cuda is available,    training on gpu


In [3]:
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms


num_workers = 0
batch_size = 25
valid_size = 0.2

transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#download the data
train_data = datasets.CIFAR10('data', train= True, download =True, transform = transform)
test_data = datasets.CIFAR10('data', train=False, download = True, transform = transform)



Files already downloaded and verified
Files already downloaded and verified


In [7]:
import numpy as np

def split_indices(total_num_images, validation_patch):
    validation_indices = int(total_num_images*validation_patch)
    shuffle_indices = np.random.permutation(total_num_images)
    return shuffle_indices[validation_indices:], shuffle_indices[:validation_indices]
    

In [8]:
train_indices, valid_indices = split_indices(len(train_data), valid_size)
print(len(train_indices), len(valid_indices))

40000 10000


In [9]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

#load train and valid data 
train_sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(train_data, batch_size=batch_size, sampler = train_sampler, num_workers=num_workers)

valid_sampler =SubsetRandomSampler(valid_indices)
valid_loader = DataLoader(train_data, batch_size = batch_size, sampler = valid_sampler, num_workers=num_workers)


#number of classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #first conv layer
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding=1)
        #maxpool layyer
        self.maxpool1 = nn.MaxPool2d(2,2)
        #2nd conv layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 1, padding =1)
        self.maxpool2 = nn.MaxPool2d(2,2)
        #3rd conv layer
        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3, stride=1, padding=1)
        self.maxpool3 = nn.MaxPool2d(2,2)
        #4th conv layer
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.maxpool4 =nn.MaxPool2d(2,2)
        
        #fully connected linear_layer1
        self.fc1 = nn.Linear(2 * 2 * 128, 256)
        #2nd linear layer
        self.fc2 = nn.Linear(256, 64)
        self.dropout = nn.Dropout(0.25)
        #output layer
        self.fc3 = nn.Linear(64, num_classes)
        
        
        
    def forward(self, x):
        
    #sequentially adding activation function to each layer and parsing forward
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
         
        x = F.relu(self.conv2(x))
        x = self.maxpool2(x)
        #adding dropout layer to avoid overfitting
        x = self.dropout(x)
        
        x = F.relu(self.conv3(x))
        x = self.maxpool3(x)
        
        x = F.relu(self.conv4(x))
        x= self.maxpool4(x)
        #dropout layer
        x = self.dropout(x)
        
        #flatten the input as nn.linear expects a vector
        x = x.view(-1, 2*2*128)
        
        x = F.relu(self.fc1(x))
        
        x = F.relu(self.fc2(x))
        #dropout layer
        x = self.dropout(x)
        
        #final ouput layer
        x = self.fc3(x)
        
        return x
    
model = Net()
print(model)


if train_on_gpu:
    model.cuda()
        
        
        

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [16]:
import torch.optim as optim

#specifing loss function and optimizer

criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [9]:
### TRANINIG THE NETWORK

n_epochs = 80
#minimum_valid_loss = np.Inf

for epoch in range(1, n_epochs+1):
    
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()
    
    for data, labels in train_loader:
        if train_on_gpu:
            data, labels = data.cuda(), labels.cuda()
            #initializing gradients zero
            optimizer.zero_grad()
            #forward propagate 
            output = model(data)
            #loss
            loss = criterion(output, labels)
            #backpropogating 
            loss.backward()
            #update parameters
            optimizer.step()
            
            #calculating the training loss
            train_loss += loss.item()*data.size(0)
            
    
    #validating the model
    model.eval()
    
    for data, labels in valid_loader:
        if train_on_gpu:
            data, labels = data.cuda(), labels.cuda()
            
            output = model(data)
            loss = criterion(output, labels)
            
            valid_loss += loss.item()*data.size(0)
            
    
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    
    
    ### print the losses accordingly for each epoch
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                        epoch, train_loss, valid_loss))
    
    #saving the model for low validation loss
    '''if valid_loss <= minimum_valid_loss:
        print('Validation loss is still decreasing  ({:.6f} --> {:.6f}). Saving our model'.format(
          minimum_valid_loss, valid_loss,))
        torch.save(model.state_dict(), 'cifar_cnn_model_pytorch.pt')
        vaild_loss = minimum_valid_loss'''
    
    torch.save(model, 'cifar_cnn_model_pytorch_epoch80.pt')
            

Epoch: 1 	Training Loss: 2.302924 	Validation Loss: 2.301638
Epoch: 2 	Training Loss: 2.296715 	Validation Loss: 2.277814
Epoch: 3 	Training Loss: 2.100834 	Validation Loss: 1.954992
Epoch: 4 	Training Loss: 1.890775 	Validation Loss: 1.798832
Epoch: 5 	Training Loss: 1.768560 	Validation Loss: 1.678166
Epoch: 6 	Training Loss: 1.680684 	Validation Loss: 1.576715
Epoch: 7 	Training Loss: 1.595449 	Validation Loss: 1.513000
Epoch: 8 	Training Loss: 1.518129 	Validation Loss: 1.431918
Epoch: 9 	Training Loss: 1.460270 	Validation Loss: 1.383956
Epoch: 10 	Training Loss: 1.406158 	Validation Loss: 1.305016
Epoch: 11 	Training Loss: 1.350966 	Validation Loss: 1.276302
Epoch: 12 	Training Loss: 1.298606 	Validation Loss: 1.207205
Epoch: 13 	Training Loss: 1.251252 	Validation Loss: 1.132711
Epoch: 14 	Training Loss: 1.204121 	Validation Loss: 1.216568
Epoch: 15 	Training Loss: 1.163088 	Validation Loss: 1.068380
Epoch: 16 	Training Loss: 1.119907 	Validation Loss: 1.021274
Epoch: 17 	Traini

In [27]:
model = Net()
model.load_state_dict(torch.load('cifar_cnn_model_pytorch_epoch80.pt'))
model.eval()
model.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [31]:
test_loader = DataLoader(test_data, batch_size = batch_size, num_workers=num_workers)

test_loss = 0

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range (10))

model.eval()

for data, labels in test_loader:
    data, labels = data.cuda(), labels.cuda()
    
    output = model(data)
    loss = criterion(output, labels)
    test_loss += loss.item()*data.size(0)
    
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    
    for i in range(batch_size):
        label = labels.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
    

Test Loss: 0.712781

Test Accuracy of airplane: 80% (808/1000)
Test Accuracy of automobile: 86% (861/1000)
Test Accuracy of  bird: 62% (622/1000)
Test Accuracy of   cat: 58% (588/1000)
Test Accuracy of  deer: 73% (735/1000)
Test Accuracy of   dog: 68% (682/1000)
Test Accuracy of  frog: 91% (912/1000)
Test Accuracy of horse: 79% (798/1000)
Test Accuracy of  ship: 89% (891/1000)
Test Accuracy of truck: 87% (878/1000)

Test Accuracy (Overall): 77% (7775/10000)
